In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import yfinance as yf
import gspread

SPREADSHEET_KEY=os.environ['SPREADSHEET_KEY']
ROBINHOOD_BROKERAGE_FILE_PATH='/Users/jaredyu/Desktop/finances/finance_tracker_app/data/investing_statements/robinhood/brokerage'
ROBINHOOD_TRADITIONAL_IRA_FILE_PATH='/Users/jaredyu/Desktop/finances/finance_tracker_app/data/investing_statements/robinhood/traditional_ira/'

In [2]:
# need to open with Numbers, remove the hanging cell, and then export as CSV with utf-8 encoding
brokerage_df = pd.read_csv(
    os.path.join(
        ROBINHOOD_BROKERAGE_FILE_PATH,
        '20161018_20240530.csv'
        # '20201201_20240526.csv'
    )
)

# subset data to simplify analysis (start date of trading 2021/1/1)
brokerage_df = brokerage_df.loc[pd.to_datetime(brokerage_df['Activity Date']) > datetime(2021, 1, 1)].copy()

traditional_ira_df = pd.read_csv(
    os.path.join(
        ROBINHOOD_TRADITIONAL_IRA_FILE_PATH,
        '20230116_20240526.csv'
    )
)

In [3]:
# get latest prices for all instruments
df_list = [
    brokerage_df,
    traditional_ira_df,
]
instrument_list = []
for df in df_list:
    instrument_list.extend(list(set(df['Instrument'])))
instrument_list = [i for i in instrument_list if str(i) != 'nan']
instrument_list = list(set(instrument_list))

latest_bid_list = []
for symbol in instrument_list:
    latest_bid = yf.Ticker(symbol).info['previousClose']
    latest_bid_list.append(latest_bid)
instrument_price_df = pd.DataFrame(list(zip(instrument_list, latest_bid_list)))
instrument_price_df.columns = ['SYMBOL', 'PRICE']

In [4]:
def clean_price_col(df):
    df = df.copy()
    df['Price'] = df['Price'].fillna('')
    df['Price'] = df['Price'].apply(lambda x: x.replace('$', '').replace(',', ''))
    df.loc[df['Price'] == '', 'Price'] = np.NaN
    return df
    
def dollar_string_to_float(df, col):
    """
    Convert the $100.00 string to a float.
    """
    return df[col] \
        .replace('[\$,]', '', regex=True) \
        .replace('\(', '-', regex=True) \
        .replace('\)', '', regex=True) \
        .astype(float)

def clean_brokerage_account_table(df):
    brokerage_df = df.copy(deep=True)
    df['Amount'] = dollar_string_to_float(
        df=df,
        col='Amount'
    )

    # calculate net deposits (ACH transfers)
    ach_df = df.loc[
        (df['Description'].str.contains('ACH')) &
        (df['Trans Code'] == 'ACH'),
    ].copy(deep=True)
    net_deposits = round(ach_df['Amount'].sum(), 2)

    # calculate the net interest payments
    interest_payments_df = df.loc[
        (df['Description'].str.contains('Interest Payment')) &
        (df['Trans Code'].str.contains('INT'))
    ].copy(deep=True)
    net_interest_payments = round(interest_payments_df['Amount'].sum(), 2)

    # Robinhood Gold fee
    gold_fee = df.loc[df['Description'].str.contains('Gold Fee'), 'Amount'].sum()

    # calculate the stock lending
    total_stock_lending = df.loc[
        (df['Description'] == 'Stock Lending') &
        (df['Trans Code'] == 'SLIP'),
        'Amount'
    ].sum().round(2)

    # get the stock purchases/sales -> returns
    stock_purchases = df.loc[
        ~(df['Description'].str.contains('Dividend')) &
        ~(df['Description'].str.contains('Option')) &
        (df['Trans Code'] == 'Buy'),
        'Amount'
    ].sum().round(2)
    stock_sales = df.loc[
        ~(df['Description'].str.contains('Dividend')) &
        ~(df['Description'].str.contains('Option')) &
        (df['Trans Code'] == 'Sell'),
        'Amount'
    ].sum().round(2)
    investment_returns = round(stock_sales + stock_purchases, 2)

    # dividend payments
    total_reinvested_dividends = df.loc[df['Description'].str.contains('Dividend Reinvestment'), 'Amount'].sum().round(2)
    total_dividends = df.loc[df['Trans Code'].str.contains('DIV'), 'Amount'].sum().round(2)
    net_dividend_payments = total_dividends + total_reinvested_dividends

    # subset the options data
    options_df = df.loc[
        df['Description'] \
            .str \
            .contains(
                '|'.join(
                    [
                        'Call',
                        'Put',
                        'Option'
                    ]
                )
            )   
    ].copy(deep=True)

    # sort the data from earliest to latest
    options_df['Datetime'] = pd.to_datetime(options_df['Process Date'])
    options_df.sort_values(by='Datetime', ascending=True, inplace=True)
    options_df.reset_index(drop=True, inplace=True)

    # clean the description
    options_df['description_split'] = options_df['Description'].str.split('\n')
    options_df['description_split_length'] = options_df['description_split'].apply(lambda x: len(x))

    option_assigned_df = options_df.loc[
        options_df['description_split_length'] == 3,
        'Description'
    ].str.split('\n').apply(lambda x: x[2])
    option_assigned_list = option_assigned_df.values.tolist()

    # get the total options return
    options_df.loc[
        option_assigned_df.index.tolist(),
        'Description'
    ] = option_assigned_list
    options_return = options_df['Amount'].sum().round(2)

    # clean the activity column
    options_trans_code_mapping_dict = {
        'STO': 'Sell Premium',
        'OEXP': 'Option Expired',
        'OASGN': 'Option Assigned',
        'Buy': 'Buy',
        'Sell': 'Sell',
    }
    options_df['Trans Code'] = options_df['Trans Code'].map(options_trans_code_mapping_dict)
    options_df = options_df[['Process Date', 'Instrument', 'Description', 'Trans Code', 'Quantity', 'Price', 'Amount']].copy(deep=True)
    options_df.rename(columns={'Trans Code': 'Activity'}, inplace=True)

    options_df['Quantity'] = options_df['Quantity'].astype(int)
    total_premium_sold = options_df.loc[options_df['Activity'] == 'Sell Premium', 'Amount'].sum()

    # clean up nulls and save
    options_df = clean_price_col(options_df)
    options_df.replace({np.nan: None}, inplace=True)

    # subset out the options data
    brokerage_df = df.loc[
        ~df['Description'] \
            .str \
            .contains(
                '|'.join(
                    [
                        'Call',
                        'Put',
                        'Option'
                    ]
                )
            )   
    ].copy(deep=True)

    # sort the data from earliest to latest
    brokerage_df['Datetime'] = pd.to_datetime(brokerage_df['Process Date'])
    brokerage_df.sort_values(by='Datetime', ascending=True, inplace=True)
    brokerage_df.reset_index(drop=True, inplace=True)
    brokerage_df.drop('Datetime', axis=1, inplace=True)

    # clean dividend descriptions
    brokerage_df.loc[
        brokerage_df['Description'].str.contains('Cash Div'),
        'Description'
    ] = 'Cash Dividend'
    brokerage_df.loc[
        brokerage_df['Description'].str.contains('Manufactured Div'),
        'Description'
    ] = 'Manufactured Dividend'
    brokerage_df.loc[
        brokerage_df['Description'].str.contains('Dividend Reinvestment'),
        'Description'
    ] = 'Dividend Reinvestment'

    # clean the remaining \nCUSIP descriptions
    cusip_df = brokerage_df.loc[
        brokerage_df['Description'].str.contains('\nCUSIP')
    ]
    cusip_clean_description_list = cusip_df['Description'] \
        .apply(lambda x: x.split('\nCUSIP')) \
        .apply(lambda y: y[0]) \
        .values \
        .tolist()
    brokerage_df.loc[
        cusip_df.index,
        'Description'
    ] = cusip_clean_description_list

    # clean the Trans Code and the other columns
    brokerage_code_mapping_dict = {
        'ACH': 'ACH',
        'CDIV': 'Cash Dividend',
        'MDIV': 'Manufactured Dividend',
        'SLIP': 'Stock Lending',
        'Buy': 'Buy',
        'Sell': 'Sell',
        'GOLD': 'Robinhood Gold',
        'INT': 'Interest Payment',
    }
    brokerage_df['Trans Code'] = brokerage_df['Trans Code'].map(brokerage_code_mapping_dict)
    brokerage_df = brokerage_df[
        ['Process Date', 'Instrument', 'Description', 'Trans Code', 'Quantity', 'Price', 'Amount']
    ].copy(deep=True)
    brokerage_df.rename(columns={'Trans Code': 'Activity'}, inplace=True)

    brokerage_df = clean_price_col(brokerage_df)
    brokerage_df.replace({np.nan: None}, inplace=True)

    return brokerage_df, options_df

In [5]:
def clean_traditional_ira_account_table(df):
    traditional_ira_df = df.copy()
    traditional_ira_df['Amount'] = dollar_string_to_float(
        df=traditional_ira_df,
        col='Amount'
    )

    # clean the Trans Code
    traditional_ira_code_mapping_dict = {
        'ACH': 'ACH',
        'CDIV': 'Cash Dividend',
        'MDIV': 'Manufactured Dividend',
        'SLIP': 'Stock Lending',
        'Buy': 'Buy',
        'Sell': 'Sell',
        'GOLD': 'Robinhood Gold',
        'INT': 'Interest Payment',
        'MTCH': 'IRA Match',
        'DRFRO': 'Direct Rollover',
        'CFIR': 'Contribution',
        'ACATI': 'Asset Transfer',
    }
    traditional_ira_df['Trans Code'] = traditional_ira_df['Trans Code'].map(traditional_ira_code_mapping_dict)

    # clean dividend descriptions
    traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('Cash Div'),
        'Description'
    ] = 'Cash Dividend'
    traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('Manufactured Div'),
        'Description'
    ] = 'Manufactured Dividend'
    traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('Dividend Reinvestment'),
        'Description'
    ] = 'Dividend Reinvestment'

    # clean the remaining \nCUSIP descriptions
    cusip_df = traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('\nCUSIP')
    ]
    cusip_clean_description_list = cusip_df['Description'] \
        .apply(lambda x: x.split('\nCUSIP')) \
        .apply(lambda y: y[0]) \
        .values \
        .tolist()
    traditional_ira_df.loc[
        cusip_df.index,
        'Description'
    ] = cusip_clean_description_list

    # clean the asset transfer descriptions
    traditional_ira_df.loc[
        (traditional_ira_df['Trans Code'].str.contains('Asset')) &
        (traditional_ira_df['Description'].str.contains('ACAT IN')),
        'Description'
    ] = 'Asset Transfer'

    # clean direct rollover
    traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('Direct Rollover Check Received'),
        'Description'
    ] = 'Direct Rollover Check'

    # clean IRA interest description
    traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('Interest on Contribution \(IRA Match\)'),
        'Description'
    ] = 'Robinhood IRA Match'

    # clean IRA contribution description
    traditional_ira_df.loc[
        traditional_ira_df['Description'].str.contains('Current Year Contribution'),
        'Description'
    ] = 'IRA Contribution'

    # sort the data from earliest to latest
    traditional_ira_df['Datetime'] = pd.to_datetime(traditional_ira_df['Process Date'])
    traditional_ira_df.sort_values(by='Datetime', ascending=True, inplace=True)
    traditional_ira_df.reset_index(drop=True, inplace=True)

    # clean the columns
    traditional_ira_df = traditional_ira_df[
        ['Process Date', 'Instrument', 'Description', 'Trans Code', 'Quantity', 'Price', 'Amount']
    ].copy(deep=True)
    traditional_ira_df.rename(columns={'Trans Code': 'Activity'}, inplace=True)

    traditional_ira_df = clean_price_col(traditional_ira_df)
    traditional_ira_df.replace({np.nan: None}, inplace=True)

    return traditional_ira_df

In [6]:
def add_daily_portfolio_value(df, date_col, buy_sell_col):
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(date_col, ascending=True, inplace=True)
    df['Quantity'] = df['Quantity'].astype(float)

    buy_sell_actions_list = ['Buy', 'Sell', 'Asset Transfer']
    buy_sell_df = df.loc[df[buy_sell_col].isin(buy_sell_actions_list)].copy()
    activity_date_list = sorted(list(set(buy_sell_df[date_col])))

    activity_date_tot_val_list = []
    for activity_date in activity_date_list:
        buy_sell_temp_df = buy_sell_df.loc[buy_sell_df[date_col] <= activity_date].copy()
        tot_val = 0
        instrument_list = [
            i for i in list(set(buy_sell_temp_df['Instrument'])) if type(i) == str
        ] # drop np.nan
        for instrument in instrument_list:
            instrument_df = buy_sell_temp_df.loc[
                (buy_sell_temp_df['Instrument'] == instrument) &
                (buy_sell_temp_df[buy_sell_col].isin(buy_sell_actions_list))
            ]

            num_bought = instrument_df[instrument_df[buy_sell_col].isin(['Buy', 'Asset Transfer'])]['Quantity'].sum()
            num_sold = instrument_df[instrument_df[buy_sell_col] == 'Sell']['Quantity'].sum()
            net_quantity = num_bought - num_sold

            closing_prices = yf.download(
                instrument, start=activity_date, end=activity_date + timedelta(hours=16), progress=False
            )
            closing_prices = round(closing_prices['Close'].values[0], 2)

            cur_value = net_quantity * closing_prices
            tot_val += cur_value

        tot_val = round(tot_val, 2)
        activity_date_tot_val_list.append([activity_date, tot_val])

    activity_date_tot_val_df = pd.DataFrame(activity_date_tot_val_list)
    activity_date_tot_val_df.columns = [date_col, 'Portfolio Value']

    df = pd.merge(
        left=df,
        right=activity_date_tot_val_df,
        on=date_col,
        how='left'
    )

    df[date_col] = df[date_col].astype(str)
    df = clean_price_col(df)
    df.replace({np.nan: None}, inplace=True)

    return df

In [7]:
brokerage_minus_options_df, options_df = clean_brokerage_account_table(brokerage_df)

options_premium_df = options_df.loc[options_df['Activity'] == 'Sell Premium'].copy()
options_buy_sell_df = options_df.loc[options_df['Activity'].isin(['Buy', 'Sell'])].copy()

traditional_ira_df = clean_traditional_ira_account_table(traditional_ira_df)

traditional_ira_modified_df = add_daily_portfolio_value(traditional_ira_df, 'Process Date', 'Activity')
brokerage_modified_df = add_daily_portfolio_value(brokerage_df, 'Activity Date', 'Trans Code')

In [7]:
# brokerage:
# get a cumulative balance per stock/cash
# for overview, show brokerage current value (unrealized)
# same for traditional ira

In [28]:
# brokerage_df.loc[brokerage_df['Description'] == 'ACH Deposit', 'Amount'].sum() + \
# brokerage_df.loc[brokerage_df['Description'] == 'ACH Withdrawal', 'Amount'].sum()

32975.189999999995

In [9]:
gc = gspread.service_account()
finance_tracker_db_spreadsheet = gc.open_by_key(SPREADSHEET_KEY)

robinhood_brokerage_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_brokerage_modified')
robinhood_brokerage_worksheet.update([brokerage_modified_df.columns.values.tolist()] + brokerage_modified_df.values.tolist())

robinhood_brokerage_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_traditional_ira_modified')
robinhood_brokerage_worksheet.update([traditional_ira_modified_df.columns.values.tolist()] + traditional_ira_modified_df.values.tolist())

robinhood_brokerage_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_brokerage_minus_options')
robinhood_brokerage_worksheet.update([brokerage_minus_options_df.columns.values.tolist()] + brokerage_minus_options_df.values.tolist())
# robinhood_brokerage_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})

robinhood_options_trading_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_options_trading')
robinhood_options_trading_worksheet.update([options_df.columns.values.tolist()] + options_df.values.tolist())
# robinhood_options_trading_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})

robinhood_options_trading_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_options_premium')
robinhood_options_trading_worksheet.update([options_premium_df.columns.values.tolist()] + options_premium_df.values.tolist())
# robinhood_options_trading_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})

robinhood_options_trading_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_options_buy_sell')
robinhood_options_trading_worksheet.update([options_buy_sell_df.columns.values.tolist()] + options_buy_sell_df.values.tolist())
# robinhood_options_trading_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})

robinhood_traditional_ira_worksheet = finance_tracker_db_spreadsheet.worksheet('robinhood_traditional_ira')
robinhood_traditional_ira_worksheet.update([traditional_ira_df.columns.values.tolist()] + traditional_ira_df.values.tolist())
# robinhood_traditional_ira_worksheet.format("A:A", {"numberFormat": {"type": "DATE_TIME"}})

{'spreadsheetId': '1CAyyf2kr-pS7LNX1a_0ithw6niL3Js3K4ZEOlwDViZY',
 'updatedRange': 'robinhood_traditional_ira!A1:G53',
 'updatedRows': 53,
 'updatedColumns': 7,
 'updatedCells': 285}